In [ ]:
!unzip -q "/content/drive/MyDrive/Individual_Research_Project/data/words.zip" -d /content/data/

In [ ]:
!find /content/data/words -type f -name "*.png" | wc -l

45410


In [ ]:
!cp "/content/drive/MyDrive/Individual_Research_Project/data/full_words_clean.txt" /content/data/

In [ ]:
!pip install torchvision opencv-python

In [ ]:
import os
import torch
from torch.nn import CTCLoss
from torch.utils.data import DataLoader
from torch.optim import Adam
from utils.dataset import HandwritingDataset
from utils.label_encoder import LabelEncoder
from utils.crnn_model import CRNN

# Config
img_dir = "/content/data/words"
csv_file = "/content/data/full_words_clean.txt"
batch_size = 4
num_workers = 2
epochs = 10
lr = 0.001
img_height = 32

# Drive model checkpoint directory
drive_model_dir = "/content/drive/MyDrive/Individual_Research_Project/models/"
os.makedirs(drive_model_dir, exist_ok=True)

# Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load label encoder and model
encoder = LabelEncoder()
vocab_size = encoder.get_vocab_size()
model = CRNN(img_height=img_height, num_classes=vocab_size).to(device)

# Loss and optimizer
criterion = CTCLoss(blank=0, zero_infinity=True)
optimizer = Adam(model.parameters(), lr=lr)

# Load dataset
dataset = HandwritingDataset(csv_file, img_dir)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, texts in data_loader:
        targets = [encoder.encode(t) for t in texts]
        target_lengths = torch.IntTensor([len(t) for t in targets])
        targets = torch.IntTensor([i for seq in targets for i in seq])

        images = images.to(device)
        targets = targets.to(device)
        target_lengths = target_lengths.to(device)

        output = model(images)
        input_lengths = torch.full(
            size=(output.size(0),),
            fill_value=output.size(1),
            dtype=torch.long
        ).to(device)

        loss = criterion(output.log_softmax(2).transpose(0, 1), targets, input_lengths, target_lengths)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(data_loader):.4f}")

    # Save checkpoint every 5 epochs
    if (epoch + 1) % 5 == 0:
        ckpt_path = os.path.join(drive_model_dir, f"crnn_epoch{epoch+1}.pth")
        torch.save(model.state_dict(), ckpt_path)
        print(f"Checkpoint saved to Drive: {ckpt_path}")

# Final save to Drive
final_model_path = os.path.join(drive_model_dir, "crnn_final.pth")
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved to Drive: {final_model_path}")

# 29 mins training time


In [ ]:
# Crop all annotated words and save label CSV

import os
import cv2
import xml.etree.ElementTree as ET
import pandas as pd

# Input folders (adjust if needed)
image_folder = "/content/drive/MyDrive/Individual_Research_Project/images"
xml_folder = "/content/drive/MyDrive/Individual_Research_Project/raw_xml"

# Output folder
output_crop_dir = "/content/drive/MyDrive/Individual_Research_Project/cropped_prescription_words"
os.makedirs(output_crop_dir, exist_ok=True)

# Prepare label list
cropped_data = []

# Loop through all XML files
xml_files = sorted([f for f in os.listdir(xml_folder) if f.endswith(".xml")])

for xml_file in xml_files:
    base_name = os.path.splitext(xml_file)[0]
    image_path = os.path.join(image_folder, base_name + ".jpg")
    xml_path = os.path.join(xml_folder, xml_file)

    if not os.path.exists(image_path):
        continue

    image = cv2.imread(image_path)
    if image is None:
        continue

    tree = ET.parse(xml_path)
    root = tree.getroot()

    for i, obj in enumerate(root.findall("object")):
        label = obj.find("name").text.strip()
        bndbox = obj.find("bndbox")
        xmin = int(float(bndbox.find("xmin").text)) - 10
        ymin = int(float(bndbox.find("ymin").text)) - 10
        xmax = int(float(bndbox.find("xmax").text)) + 10
        ymax = int(float(bndbox.find("ymax").text)) + 10

        # Clip bounds to image size
        xmin = max(0, xmin)
        ymin = max(0, ymin)
        xmax = min(image.shape[1], xmax)
        ymax = min(image.shape[0], ymax)

        crop = image[ymin:ymax, xmin:xmax]
        crop_filename = f"{base_name}_{i}_{label}.png"
        crop_path = os.path.join(output_crop_dir, crop_filename)
        cv2.imwrite(crop_path, crop)

        cropped_data.append({
            "image_path": crop_path,
            "label": label
        })

# Save to CSV
csv_output_path = "/content/drive/MyDrive/Individual_Research_Project/prescription_words_dataset.csv"
df = pd.DataFrame(cropped_data)
df.to_csv(csv_output_path, index=False)

print(f"Saved {len(df)} cropped word images and labels to CSV:")
print(csv_output_path)


Install Dependencies + Import Everything

In [ ]:
!pip install torchvision
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image, ImageOps
from torch import nn
from torch.optim import Adam


Define Preprocessing + Dataset Loader

In [ ]:
def resize_and_pad(img, size=(128, 32)):
    img = img.convert("L")
    w, h = img.size
    scale = size[1] / h
    new_w = int(w * scale)
    img = img.resize((new_w, size[1]), Image.BILINEAR)
    if new_w > size[0]:
        img = img.resize(size, Image.BILINEAR)
    else:
        delta_w = size[0] - new_w
        padding = (delta_w // 2, 0, delta_w - delta_w // 2, 0)
        img = ImageOps.expand(img, padding, fill=255)
    return img

class PrescriptionWordDataset(Dataset):
    def __init__(self, csv_file, img_size=(128, 32)):
        self.data = pd.read_csv(csv_file)
        self.img_size = img_size
        self.transform = transforms.Compose([
            transforms.Lambda(lambda img: resize_and_pad(img, size=img_size)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        self.encoder = LabelEncoder()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img = Image.open(row['image_path']).convert("RGB")
        img = self.transform(img)
        label = self.encoder.encode(row['label'])
        return img, torch.tensor(label), len(label)


Define LabelEncoder + CRNN Model

In [ ]:
class LabelEncoder:
    def __init__(self):
        self.characters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-.,:/()%"
        self.char2idx = {char: idx + 1 for idx, char in enumerate(self.characters)}  # 0 = blank
        self.idx2char = {idx + 1: char for idx, char in enumerate(self.characters)}
        self.blank = 0

    def encode(self, text):
        return [self.char2idx[char] for char in text if char in self.char2idx]

    def decode(self, indices):
        return ''.join([self.idx2char.get(idx, '') for idx in indices if idx != self.blank])

    def vocab_size(self):
        return len(self.char2idx) + 1

class CRNN(nn.Module):
    def __init__(self, img_height, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(512, 512, 2, 1, 0), nn.ReLU()
        )
        self.lstm1 = nn.LSTM(512, 256, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(512, 256, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        x = x.squeeze(2).permute(0, 2, 1)  # (batch, width, channels)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        return self.fc(x)


 Train the Fine-Tuned Model

In [ ]:
# Clean the CSV to remove any broken or missing image paths
df = pd.read_csv(csv_file)
df = df[df['image_path'].apply(lambda path: os.path.exists(path))]
df.to_csv(csv_file, index=False)
print(f"Cleaned dataset: {len(df)} valid image paths")


Cleaned dataset: 6479 valid image paths


In [ ]:
# Training Loop for Fine-Tuning CRNN with Prescription Words
from torch.nn import CTCLoss
from torch.optim import Adam

# Hyperparams
batch_size = 16
epochs = 10
lr = 0.001
csv_file = "/content/drive/MyDrive/Individual_Research_Project/prescription_words_dataset.csv"

# Re-setup everything cleanly
encoder = LabelEncoder()
dataset = PrescriptionWordDataset(csv_file)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CRNN(img_height=32, num_classes=encoder.vocab_size()).to(device)
criterion = CTCLoss(blank=0, zero_infinity=True)
optimizer = Adam(model.parameters(), lr=lr)

# Training
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        batch_images, batch_targets, batch_lengths = [], [], []

        for img, label_tensor, label_len in batch:
            batch_images.append(img)
            batch_targets.extend(label_tensor.tolist())  # flatten
            batch_lengths.append(label_len)

        # Stack everything
        images = torch.stack(batch_images).to(device)
        targets = torch.tensor(batch_targets, dtype=torch.long).to(device)
        target_lengths = torch.tensor(batch_lengths, dtype=torch.long).to(device)

        output = model(images)  # (batch, time, classes)
        seq_len = output.size(1)
        input_lengths = torch.full(size=(images.size(0),), fill_value=seq_len, dtype=torch.long).to(device)

        output = output.log_softmax(2).permute(1, 0, 2)  # (time, batch, classes)

        loss = criterion(output, targets, input_lengths, target_lengths)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss / len(dataloader):.4f}")


Epoch 1/10 | Loss: 3.5441
Epoch 2/10 | Loss: 2.7802
Epoch 3/10 | Loss: 2.5672
Epoch 4/10 | Loss: 2.4116
Epoch 5/10 | Loss: 2.2880
Epoch 6/10 | Loss: 2.1971
Epoch 7/10 | Loss: 2.0975
Epoch 8/10 | Loss: 2.0078
Epoch 9/10 | Loss: 1.9007
Epoch 10/10 | Loss: 1.7840


training time 2h 32m

In [ ]:
# Save your fine-tuned model to Drive
save_path = "/content/drive/MyDrive/Individual_Research_Project/models/crnn_finetuned.pth"
torch.save(model.state_dict(), save_path)
print("Model saved to:", save_path)


Model saved to: /content/drive/MyDrive/Individual_Research_Project/models/crnn_finetuned.pth


In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
import pandas as pd
import torch
from torch import nn
from torchvision import transforms
from PIL import Image, ImageOps
from difflib import get_close_matches

# ==== Paths ====
image_folder = "/content/drive/MyDrive/Individual_Research_Project/images"
xml_folder = "/content/drive/MyDrive/Individual_Research_Project/raw_xml"
model_path = "/content/drive/MyDrive/Individual_Research_Project/models/crnn_finetuned.pth"
kb_path = "/content/drive/MyDrive/Individual_Research_Project/knowledge_base.xlsx"

# ==== LabelEncoder ====
class LabelEncoder:
    def __init__(self):
        self.characters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-.,:/()%"
        self.char2idx = {char: idx + 1 for idx, char in enumerate(self.characters)}  # 0 = blank
        self.idx2char = {idx + 1: char for idx, char in enumerate(self.characters)}
        self.blank = 0

    def encode(self, text):
        return [self.char2idx.get(char, 0) for char in text]

    def decode(self, indices):
        return ''.join([self.idx2char.get(idx, '') for idx in indices if idx != self.blank])

    def vocab_size(self):
        return len(self.char2idx) + 1

# ==== CRNN Model ====
class CRNN(nn.Module):
    def __init__(self, img_height, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(512, 512, 2, 1, 0), nn.ReLU()
        )
        self.lstm1 = nn.LSTM(512, 256, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(512, 256, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        x = x.squeeze(2).permute(0, 2, 1)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        return self.fc(x)

# ==== Preprocessing Function ====
def resize_and_pad(img, size=(128, 32)):
    img = img.convert("L")
    w, h = img.size
    scale = size[1] / h
    new_w = int(w * scale)
    img = img.resize((new_w, size[1]), Image.BILINEAR)
    if new_w > size[0]:
        img = img.resize(size, Image.BILINEAR)
    else:
        delta_w = size[0] - new_w
        padding = (delta_w // 2, 0, delta_w - delta_w // 2, 0)
        img = ImageOps.expand(img, padding, fill=255)
    return img

def predict_word(image, model, encoder, device):
    image = resize_and_pad(Image.fromarray(image))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        output = output.softmax(2)
        pred_indices = torch.argmax(output, dim=2).squeeze().tolist()
        return encoder.decode(pred_indices).strip()

# ==== Load Model ====
encoder = LabelEncoder()
model = CRNN(img_height=32, num_classes=encoder.vocab_size())
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.eval()

# ==== Load Knowledge Base ====
kb = pd.read_excel(kb_path)
kb_drugs = kb[kb["Category"].str.lower() == "drug name"]["Entry"].dropna().str.lower().unique()

# ==== Process a Single File ====
results = []
filename = "1000"
image_path = os.path.join(image_folder, filename + ".jpg")
xml_path = os.path.join(xml_folder, filename + ".xml")

if os.path.exists(image_path) and os.path.exists(xml_path):
    image = cv2.imread(image_path)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    for obj in root.findall('object'):
        bndbox = obj.find('bndbox')
        xmin = int(float(bndbox.find('xmin').text)) - 10
        ymin = int(float(bndbox.find('ymin').text)) - 10
        xmax = int(float(bndbox.find('xmax').text)) + 10
        ymax = int(float(bndbox.find('ymax').text)) + 10

        xmin = max(0, xmin)
        ymin = max(0, ymin)
        xmax = min(image.shape[1], xmax)
        ymax = min(image.shape[0], ymax)

        crop = image[ymin:ymax, xmin:xmax]
        pred = predict_word(crop, model, encoder, device="cpu")

        match = get_close_matches(pred.lower(), kb_drugs, n=1, cutoff=0.7)
        if match:
            info = kb[(kb["Entry"].str.lower() == match[0]) & (kb["Category"].str.lower() == "drug name")].iloc[0]
            results.append({
                "Predicted": pred,
                "Matched Drug": info["Entry"],
                "Usage": info.get("Common Usage", "N/A"),
                "Side Effects": info.get("Side Effects", "N/A"),
                "Cautions": info.get("Cautions", "N/A")
            })
        else:
            results.append({
                "Predicted": pred,
                "Matched Drug": "No Match",
                "Usage": "N/A",
                "Side Effects": "N/A",
                "Cautions": "N/A"
            })

# ==== Show Results ====
pd.DataFrame(results)


,Predicted,Matched Drug,Usage,Side Effects,Cautions
0,Coepine,No Match,N/A,N/A,N/A
1,5d,No Match,N/A,N/A,N/A
2,3.d,No Match,N/A,N/A,N/A
3,Omepol,Omeprazole,N/A,N/A,N/A
4,Ce-pone,No Match,N/A,N/A,N/A
5,10mg,No Match,N/A,N/A,N/A
6,1mg,No Match,N/A,N/A,N/A
7,3d,No Match,N/A,N/A,N/A
8,50mg,No Match,N/A,N/A,N/A
9,1d,No Match,N/A,N/A,N/A


In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
import pandas as pd
import torch
from torch import nn
from torchvision import transforms
from PIL import Image, ImageOps
from sentence_transformers import SentenceTransformer, util

# Paths
image_folder = "/content/drive/MyDrive/Individual_Research_Project/images"
xml_folder = "/content/drive/MyDrive/Individual_Research_Project/raw_xml"
model_path = "/content/drive/MyDrive/Individual_Research_Project/models/crnn_finetuned.pth"
kb_path = "/content/drive/MyDrive/Individual_Research_Project/knowledge_base.xlsx"
output_csv = "/content/drive/MyDrive/Individual_Research_Project/prediction_results_full.csv"

# Load knowledge base
kb = pd.read_excel(kb_path)
kb_drugs = kb[kb["Category"].str.lower() == "drug name"]["Entry"].dropna().str.lower().unique().tolist()

# Load embedding model and encode KB entries
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
kb_embeddings = embedding_model.encode(kb_drugs, convert_to_tensor=True)

# LabelEncoder
class LabelEncoder:
    def __init__(self):
        self.characters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-.,:/()%"
        self.char2idx = {char: idx + 1 for idx, char in enumerate(self.characters)}
        self.idx2char = {idx + 1: char for idx, char in enumerate(self.characters)}
        self.blank = 0

    def encode(self, text):
        return [self.char2idx.get(char, 0) for char in text]

    def decode(self, indices):
        return ''.join([self.idx2char.get(idx, '') for idx in indices if idx != self.blank])

    def vocab_size(self):
        return len(self.char2idx) + 1

# CRNN Model
class CRNN(nn.Module):
    def __init__(self, img_height, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(512, 512, 2, 1, 0), nn.ReLU()
        )
        self.lstm1 = nn.LSTM(512, 256, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(512, 256, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        x = x.squeeze(2).permute(0, 2, 1)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        return self.fc(x)

# Preprocessing
def resize_and_pad(img, size=(128, 32)):
    img = img.convert("L")
    w, h = img.size
    scale = size[1] / h
    new_w = int(w * scale)
    img = img.resize((new_w, size[1]), Image.BILINEAR)
    if new_w > size[0]:
        img = img.resize(size, Image.BILINEAR)
    else:
        delta_w = size[0] - new_w
        padding = (delta_w // 2, 0, delta_w - delta_w // 2, 0)
        img = ImageOps.expand(img, padding, fill=255)
    return img

# Prediction + Matching
def predict_word(image, model, encoder, device="cpu"):
    image = resize_and_pad(Image.fromarray(image))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = transform(image).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(image)
        output = output.softmax(2)
        pred_indices = torch.argmax(output, dim=2).squeeze().tolist()
        return encoder.decode(pred_indices).strip()

def match_with_kb(pred, kb_drugs, kb_embeddings, threshold=0.6):
    pred_emb = embedding_model.encode(pred.lower(), convert_to_tensor=True)
    sim_scores = util.cos_sim(pred_emb, kb_embeddings)[0]
    best_score = float(sim_scores.max())
    best_index = int(sim_scores.argmax())
    if best_score >= threshold:
        return kb_drugs[best_index], round(best_score, 2)
    return "No Match", "N/A"

# Load model
encoder = LabelEncoder()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CRNN(img_height=32, num_classes=encoder.vocab_size()).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# Run full prediction loop
results = []
xml_files = sorted([f for f in os.listdir(xml_folder) if f.endswith(".xml")])

for xml_file in xml_files:
    base_name = os.path.splitext(xml_file)[0]
    image_path = os.path.join(image_folder, base_name + ".jpg")
    xml_path = os.path.join(xml_folder, xml_file)
    if not os.path.exists(image_path):
        continue

    image = cv2.imread(image_path)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    for obj in root.findall('object'):
        bndbox = obj.find('bndbox')
        xmin = int(float(bndbox.find('xmin').text)) - 10
        ymin = int(float(bndbox.find('ymin').text)) - 10
        xmax = int(float(bndbox.find('xmax').text)) + 10
        ymax = int(float(bndbox.find('ymax').text)) + 10
        xmin = max(0, xmin)
        ymin = max(0, ymin)
        xmax = min(image.shape[1], xmax)
        ymax = min(image.shape[0], ymax)

        crop = image[ymin:ymax, xmin:xmax]
        pred = predict_word(crop, model, encoder, device=device)
        match, score = match_with_kb(pred, kb_drugs, kb_embeddings)
        results.append({
            "Image": base_name,
            "Predicted": pred,
            "Matched Drug": match,
            "Similarity": score
        })

# Save final results
df_full = pd.DataFrame(results)
df_full.to_csv(output_csv, index=False)
print("All predictions saved to:", output_csv)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

All predictions saved to: /content/drive/MyDrive/Individual_Research_Project/prediction_results_full.csv


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load your full KB
kb_df = pd.read_excel("/content/drive/MyDrive/Individual_Research_Project/knowledge_base.xlsx")

# Clean KB: lowercase and drop NaNs
kb_df = kb_df.dropna(subset=["Entry", "Category"])
kb_df["Entry"] = kb_df["Entry"].str.lower().str.strip()
kb_df["Category"] = kb_df["Category"].str.lower().str.strip()

# Create KB Entries + Categories
kb_entries = kb_df["Entry"].astype(str).tolist()
kb_categories = kb_df["Category"].tolist()

# Encode all KB entries
model = SentenceTransformer("all-MiniLM-L6-v2")
kb_embeddings = model.encode(kb_entries, convert_to_tensor=True)

# Load previous predictions CSV
pred_df = pd.read_csv("/content/drive/MyDrive/Individual_Research_Project/prediction_results_full.csv")
pred_list = pred_df["Predicted"].astype(str).str.lower().str.strip().tolist()
file_ids = pred_df["Image"].tolist()

# Match each prediction to best KB entry
results = []
for i, pred in enumerate(pred_list):
    pred_emb = model.encode(pred, convert_to_tensor=True)
    sim_scores = util.cos_sim(pred_emb, kb_embeddings)[0]
    best_idx = int(sim_scores.argmax())
    best_score = float(sim_scores[best_idx])

    if best_score >= 0.6:
        matched_word = kb_entries[best_idx]
        matched_category = kb_categories[best_idx]
    else:
        matched_word = "No Match"
        matched_category = "N/A"
        best_score = "N/A"

    results.append({
        "File": file_ids[i],
        "Predicted": pred,
        "Matched Entry": matched_word,
        "Category": matched_category,
        "Similarity": best_score
    })

# Save upgraded results
df_upgraded = pd.DataFrame(results)
output_path = "/content/drive/MyDrive/Individual_Research_Project/prediction_results_with_categories.csv"
df_upgraded.to_csv(output_path, index=False)

print("Updated predictions saved to:")
print(output_path)


Updated predictions saved to:
/content/drive/MyDrive/Individual_Research_Project/prediction_results_with_categories.csv


Faster RCNN

In [ ]:
!pip install -q torchvision matplotlib

In [ ]:
import os
import cv2
import torch
import xml.etree.ElementTree as ET
from PIL import Image
from torchvision.transforms import functional as F

class PrescriptionDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, xml_dir, transforms=None):
        self.img_dir = img_dir
        self.xml_dir = xml_dir
        self.transforms = transforms

        # Only keep IDs where both .jpg and .xml exist
        xml_ids = [fname[:-4] for fname in os.listdir(xml_dir) if fname.endswith(".xml")]
        self.image_ids = [
            img_id for img_id in xml_ids
            if os.path.exists(os.path.join(img_dir, img_id + ".jpg"))
        ]

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        img_path = os.path.join(self.img_dir, image_id + ".jpg")
        xml_path = os.path.join(self.xml_dir, image_id + ".xml")

        img = Image.open(img_path).convert("RGB")
        boxes = []
        labels = []

        tree = ET.parse(xml_path)
        root = tree.getroot()

        for obj in root.findall('object'):
            label = 1  # word label
            bbox = obj.find('bndbox')
            xmin = int(float(bbox.find('xmin').text))
            ymin = int(float(bbox.find('ymin').text))
            xmax = int(float(bbox.find('xmax').text))
            ymax = int(float(bbox.find('ymax').text))
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(label)

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        image_id_tensor = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": image_id_tensor,
            "area": area,
            "iscrowd": iscrowd
        }

        if self.transforms:
            img = self.transforms(img)

        return img, target


In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Subset
import torch
import torchvision.transforms as T
import time
import os

# Transforms
def get_transform(train=True):
    transforms = []
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

# Load dataset
full_dataset = PrescriptionDataset(
    img_dir="/content/drive/MyDrive/Individual_Research_Project/images",
    xml_dir="/content/drive/MyDrive/Individual_Research_Project/raw_xml",
    transforms=get_transform()
)

# Limit to 300 samples
subset_indices = list(range(min(500, len(full_dataset))))
train_dataset = Subset(full_dataset, subset_indices)
data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Load pretrained Faster R-CNN
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace classifier head
num_classes = 2  # 1 word class + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Setup
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

optimizer = torch.optim.SGD([p for p in model.parameters() if p.requires_grad],
                            lr=0.005, momentum=0.9, weight_decay=0.0005)

num_epochs = 20

# Training loop
print("Training Faster R-CNN")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start = time.time()

    for i, (images, targets) in enumerate(data_loader):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_loss += losses.item()
        print(f"Epoch {epoch+1} | Batch {i+1}/{len(data_loader)} | Loss: {losses.item():.4f}", end='\r')

    print(f"\nEpoch {epoch+1}/{num_epochs} completed. Total Loss: {total_loss:.4f} | Time: {round(time.time()-start)}s")

# Save model
save_path = "/content/drive/MyDrive/Individual_Research_Project/models/faster_rcnn_words_new_1.pth"
torch.save(model.state_dict(), save_path)
print(f"\nModel saved to: {save_path}")


Training Faster R-CNN
Epoch 1 | Batch 125/125 | Loss: 0.9574
Epoch 1/20 completed. Total Loss: 100.7493 | Time: 156s
Epoch 2 | Batch 125/125 | Loss: 1.0022
Epoch 2/20 completed. Total Loss: 71.5335 | Time: 157s
Epoch 3 | Batch 125/125 | Loss: 0.4419
Epoch 3/20 completed. Total Loss: 61.8185 | Time: 157s
Epoch 4 | Batch 125/125 | Loss: 0.3399
Epoch 4/20 completed. Total Loss: 55.3805 | Time: 156s
Epoch 5 | Batch 125/125 | Loss: 0.3702
Epoch 5/20 completed. Total Loss: 51.1093 | Time: 156s
Epoch 6 | Batch 125/125 | Loss: 0.3855
Epoch 6/20 completed. Total Loss: 47.9783 | Time: 157s
Epoch 7 | Batch 125/125 | Loss: 0.3799
Epoch 7/20 completed. Total Loss: 44.2979 | Time: 156s
Epoch 8 | Batch 125/125 | Loss: 0.4568
Epoch 8/20 completed. Total Loss: 39.7747 | Time: 156s
Epoch 9 | Batch 125/125 | Loss: 0.2504
Epoch 9/20 completed. Total Loss: 36.8115 | Time: 156s
Epoch 10 | Batch 125/125 | Loss: 0.2246
Epoch 10/20 completed. Total Loss: 34.4023 | Time: 157s
Epoch 11 | Batch 125/125 | Loss: 0.